In [13]:
import numpy as np 
import h5py
from scipy.special import j0

import struct
from os.path import getsize

import import_ipynb
from geometry import *
from parameters import *

In [14]:
class fieldfile():
    """
    A class that reads and formats the field file from GENE so that we can import the real, complex, and abs terms of the eigenfunction.
    Uses pars to construct the resolution of the field. 
    """
    def __init__(self,file,pars):
        self.pars=pars
        self.file=file
        self.set_gridcounts()
        self.set_sizes()
        self.set_datatypes()
        self.te,self.tesize=self.TimeEntry()
        self.define_arrays()
        self.redirect(self.file)

    #Opens field file to begin reading from file
    def redirect(self,file):
        self.file=file
        try: 
            close(self.f)
        except:
            pass
        self.f=open(file,'rb')
        self.tfld=[]
        self.get_timearray()
        self.reset_tinds()

    #Calls from pars, given by parameters_XXXX, to set the grid size/resolution of the field.
    def set_gridcounts(self):
        self.nx=int(self.pars['nx0'])
        self.ny=int(self.pars['nky0'])
        self.nz=int(self.pars['nz0'])

    #Sets the size of the array storing the field using pars. 
    def set_sizes(self):
        self.nfields=int(self.pars['n_fields'])
        self.intsize=4
        realsize=4+(self.pars['PRECISION']=='DOUBLE')*4
        complexsize=2*realsize
        self.entrysize=self.nx*self.ny*self.nz*complexsize
        self.leapfld=self.nfields*(self.entrysize+2*self.intsize)

    #Sets the datatype of the points being read from field file.
    def set_datatypes(self):
        try: self.bigendian=pars['ENDIANNESS']=='BIG'
        except:
            self.bigendian=False
        if self.bigendian:
            self.nprt=(np.dtype(np.float64)).newbyteorder()
            self.npct=(np.dtype(np.complex128)).newbyteorder()
        else:
            self.nprt=np.dtype(np.float64)
            self.npct=np.dtype(np.complex128)

    #Initializes the arrays for phi and apar.
    def define_arrays(self):
        self.phi3d=np.empty((self.nz,self.ny,self.nx),dtype=self.npct)
        self.apar3d=np.empty((self.nz,self.ny,self.nx),dtype=self.npct)
            
    #Gets the time information from the field file. 
    def get_timearray(self):
        for i in range(getsize(self.file)//(self.leapfld+self.tesize)):
            self.tfld.append(float(self.te.unpack(self.f.read(self.tesize))[1]))
            self.f.seek(self.leapfld,1)

    #Finds the min and max time from the field file. 
    def get_minmaxtime(self):
        if not self.tfld:
            self.get_timearray()
        return self.tfld[0],self.tfld[-1]

    #Declares the struct for phi(t).
    def TimeEntry(self):
            if self.bigendian:
                    timeentry=struct.Struct('>idi')
            else:
                    timeentry=struct.Struct('=idi')
            return timeentry,timeentry.size

    #Determines if there is some offset from the midplane. 
    def offset(self,var):
            if var in [i for i in range(self.nfields)]:
                    return self.tesize+self.tind*(self.tesize+self.leapfld)+var*(self.entrysize+2*self.intsize)+self.intsize

    #Reads and returns the field for a given time.
    def readvar(self,var):
        self.f.seek(self.offset(var))	
        var3d=np.fromfile(self.f,count=self.nx*self.ny*self.nz,dtype=self.npct).reshape(self.nz,self.ny,self.nx)
        return var3d

    #Returns a time value for a particular phi value.
    def get_tind(self):
        return self.tfld.index(self.time)

    #Sets the current time step.
    def set_time(self,time):
        self.time=time
        self.tind=self.get_tind()

    #Resets the time.
    def reset_tinds(self):
        self.tind=0
        self.ftind=[-1]*self.nfields

    #Reads phi from the field_XXXX file.
    def phi(self):
        if self.ftind[0]==self.tind:
            pass
        else:
            self.ftind[0]=self.tind
            self.phi3d=self.readvar(0)
        return self.phi3d

    #Reads apar from the field_XXXX file.
    def apar(self):
        if self.nfields>1:
            if self.ftind[1]==self.tind:
                pass
            else:
                self.ftind[1]=self.tind
                self.apar3d=self.readvar(1)
            return self.apar3d

In [15]:
def calc_kperp_omd_h5(coord, geom_h5, geom_type, geom_coeff, pars, center_only, output_B = False):
    """ 
    Calculates the kperp, omd_curv, and omd_gradB along the field line using input files from GENE. 
    """
    #Initializes variables and assigns values for calculation using pars. 
    ky = float(pars['kymin'])
    nx = int(pars['nx0'])
    nz = int(pars['nz0'])
    lx = float(pars['lx'])

    if center_only:
        ikx_grid = [0]
    else:
        ikx_grid = np.arange(- nx // 2 + 1, nx // 2 + 1)

    dkx = 2. * np.pi * float(pars['shat']) * float(ky)
    dpdx_tot = float(pars['beta']) * \
               (float(pars['omn1']) + float(pars['omt1']))

    #Calculates contributions to dpdx depending on number of particle species.
    if  int(pars['n_spec']) > 1:
        dpdx_tot = dpdx_tot + float(pars['beta']) * \
                   (float(pars['omn2']) + float(pars['omt2']))
        if int(pars['n_spec']) > 2:
           dpdx_tot = dpdx_tot + float(pars['beta']) * \
                      (float(pars['omn3']) + float(pars['omt3']))

    #Checks kx_center, if not assigned by GENE, assumed to be 0 (midplane). 
    if 'kx_center' in pars:
        kx_center = float(pars['kx_center'])
    else:
        kx_center = 0.

    #Calculates curvature.
    Kx, Ky = read_curv_coeff(geom_type, geom_coeff, False)
    Kx_h5, Ky_h5 = read_curv_coeff_h5(geom_h5, coord, geom_type, False)

    #Reads in magnetic geometry coefficients. 
    ggxx = geom_coeff['ggxx'].astype(float)
    ggxy = geom_coeff['ggxy'].astype(float)
    ggyy = geom_coeff['ggyy'].astype(float)
    gBfield = geom_coeff['gBfield'].astype(float)

    #Initializes kperp, omd_curv, and omdgradB, depending on center_only coordinates or not. 
    if center_only:
        kperp = np.zeros(nz,dtype='float128')
        omd_curv = np.zeros(nz,dtype='float128')
        omd_gradB = np.zeros(nz,dtype='float128')
    else:
        kperp = np.zeros(nx*nz,dtype='float128')
        omd_curv = np.zeros(nx*nz,dtype='float128')
        omd_gradB = np.zeros(nx*nz,dtype='float128')

    #Initializes and populates kx_grid.
    kx_grid = []

    for i in ikx_grid:
        kx = i*dkx+kx_center
        this_kperp = np.sqrt(ggxx*kx**2+2.*ggxy*kx*ky+ggyy*ky**2)
        this_omegad_gradB = -(Kx*kx+Ky*ky)/gBfield
        this_omegad_curv = this_omegad_gradB + ky * float(pars['dpdx_pm'])/gBfield**2/2.

        kperp[(i-ikx_grid[0])*nz:(i-ikx_grid[0]+1)*nz]=this_kperp
        omd_curv[(i-ikx_grid[0])*nz:(i-ikx_grid[0]+1)*nz]=this_omegad_curv
        omd_gradB[(i-ikx_grid[0])*nz:(i-ikx_grid[0]+1)*nz]=this_omegad_gradB
        kx_grid.append(kx)

    #Reads in kx_grid and ky_grid from coordinate file. 
    kxgrid_h5 = coord.get('/coord/kx')[()]
    kygrid_h5_tmp = coord.get('/coord/ky')[()]
    kygrid_h5 = kygrid_h5_tmp[0]

    #Reads in magnetic field from magn_geometry_XXXX file. 
    gxx_h5 = geom_h5.get('/metric/g^xx')[()]
    gxy_h5 = geom_h5.get('/metric/g^xy')[()]
    gyy_h5 = geom_h5.get('/metric/g^yy')[()]
    Bfield_h5 = geom_h5.get('Bfield_terms/Bfield')[()]

    #Calculates kperp, omd_curve, and omd_gradB.
    if center_only:
        kperp_h5 = np.sqrt(gyy_h5 * kygrid_h5**2)
        omd_gradB_h5 = -(Ky_h5 * kygrid_h5) / Bfield_h5
        omd_curv_h5 = this_omegad_gradB + \
                           kygrid_h5 * float(pars['dpdx_pm']) / Bfield_h5**2 / 2.
        return kperp_h5, omd_curv_h5, omd_gradB_h5
    else:
        kperp_h5 = np.zeros(nx*nz,dtype='float128')
        omd_curv_h5 = np.zeros(nx*nz,dtype='float128')
        omd_gradB_h5 = np.zeros(nx*nz,dtype='float128')
        B_h5 = np.zeros(nx*nz,dtype='float128')
    kxgrid_h5 = np.append(kxgrid_h5[(nx//2 + 1):], kxgrid_h5[:(nx//2 + 1)])

    if float(pars['shat']) < 0 :
        kxgrid_h5 = kxgrid_h5[::-1]

    #Populates arrays for values of kperp, omd_curv, and omd_gradB.
    for i, kx in enumerate(kxgrid_h5):
        this_kperp = np.sqrt(gxx_h5 * kx**2 + 2. * gxy_h5 * kx * kygrid_h5 + gyy_h5 * kygrid_h5**2)
        this_omegad_gradB = -(Kx_h5 * kx + Ky_h5 * kygrid_h5) / Bfield_h5
        this_omegad_curv = this_omegad_gradB + kygrid_h5 * float(pars['dpdx_pm']) / Bfield_h5**2 / 2.

        kperp_h5[i*nz:(i+1)*nz]=this_kperp
        omd_curv_h5[i*nz:(i+1)*nz]=this_omegad_curv
        omd_gradB_h5[i*nz:(i+1)*nz]=this_omegad_gradB
        B_h5[i*nz:(i+1)*nz]=Bfield_h5

    #Determines output of magnetic field if it's asked for. 
    if output_B:
        return kperp_h5, omd_curv_h5, omd_gradB_h5, B_h5
    else:
        return kperp_h5, omd_curv_h5, omd_gradB_h5

In [16]:
def eigenfunctions_from_field_file(pars,suffix,center_only,smooth_field = False, scale_field = False):
    """ 
    Imports the eigenfunction from the field_XXXX file and allocates it to an array.
    """
    #Reads in the field file from field_XXXX.
    field = fieldfile('field'+suffix,pars)

    #Calls grid size from pars.
    nz = int(field.nz)
    nx = int(field.nx)

    #Initialzes arrays depending on if in center_only coordinates.
    if center_only:
        ikx_grid = [0]
        phi = np.zeros(nz,dtype='complex128')
        apar = np.zeros(nz,dtype='complex128')
    else:
        ikx_grid = np.arange(-nx//2+1,nx//2+1)
        phi = np.zeros(nx*nz,dtype='complex128')
        apar = np.zeros(nx*nz,dtype='complex128')

    #Curvature contribution calculation.
    if 'n0_global' in pars:
        phase_fac = -np.e**(-2.0*np.pi*(0.0+1.0J)*int(pars['n0_global']) * float(pars['q0']))
    else:
        phase_fac = -1.0
    
    
    if float(pars['shat']) > 0.:
        for i in ikx_grid:
            this_phi = field.phi()[:,0,i]*phase_fac**i
            phi[(i-ikx_grid[0])*nz:(i-ikx_grid[0]+1)*nz]=this_phi
            if int(pars['n_fields']) > 1 and float(pars['beta']) !=0:
                this_apar = field.apar()[:,0,i]*phase_fac**i
                apar[(i-ikx_grid[0])*nz:(i-ikx_grid[0]+1)*nz]=this_apar
    else:
        for i in ikx_grid:
            this_phi = field.phi()[:,0,-i]*phase_fac**i
            phi[(i-ikx_grid[0])*nz:(i-ikx_grid[0]+1)*nz]=this_phi
            if int(pars['n_fields']) > 1 and float(pars['beta']) !=0.:
                this_apar = field.apar()[:,0,-i]*phase_fac**i
                apar[(i-ikx_grid[0])*nz:(i-ikx_grid[0]+1)*nz]=this_apar

    # Normalize phi and apar to 1. 
    if scale_field:
        phi = phi/np.max(abs(field.phi()[:,0,:]))
        if int(pars['n_fields']) > 1 and float(pars['beta']) !=0:
            apar = apar/np.max(abs(field.apar()[:,0,:]))
            
    return phi, apar

In [17]:
def eigenfunction_average_bessel(z_grid,jacobian,kperp,omega_d,field):
    """
    Calculates the eigenfunction-average of kperp, omega_d, and the bessel factor.
    """
    #Defines the bessel factor
    alpha = 2./3.
    bessel_factor = 1. / np.sqrt(1. + 2. * (kperp**2 + np.pi * alpha * kperp**4) /
                    (1. + alpha * kperp**2))

    #Initializes values for eigen-averaged kperp**2. 
    ave_kperp2 = 0.
    denom = 0.

    #Calculates the eigenfunction average of kperp**2 and kperp.
    for i in np.arange(len(field)-1):
        ave_kperp2 = ave_kperp2 + (kperp[i]**2*abs(field[i])**2 +\
            kperp[i+1]**2*abs(field[i+1])**2)/2.*\
            (z_grid[i+1]-z_grid[i])/jacobian[i]
        denom = denom + (abs(field[i])**2 +abs(field[i+1])**2)/2.*\
            (z_grid[i+1]-z_grid[i])/jacobian[i]

    ave_kperp2 = ave_kperp2/denom
    ave_kperp = np.sqrt(ave_kperp2)

    #Initializes values for eigen-averaged omega_d.
    ave_omegad = 0.
    denom = 0.
    denom2 = 0.

    #Calculates the eigenfunction average of omega_d.
    for i in np.arange(len(field)-1):
        ave_omegad = ave_omegad + (omega_d[i]*abs(field[i])**2 * bessel_factor[i]+\
            omega_d[i+1]*abs(field[i+1])**2 * bessel_factor[i+1])/2.*\
            (z_grid[i+1]-z_grid[i])/jacobian[i]
        denom = denom + (abs(field[i])**2 * bessel_factor[i] \
            +abs(field[i+1])**2 * bessel_factor[i+1])/2.*\
            (z_grid[i+1]-z_grid[i])/jacobian[i]
        denom2 = denom2 + (abs(field[i])**2 + abs(field[i+1])**2)/2.*\
            (z_grid[i+1]-z_grid[i])/jacobian[i]

    ave_omegad = ave_omegad/denom
    ave_bessel_factor = denom / denom2

    return ave_kperp, ave_omegad, ave_bessel_factor

In [18]:
def bessel_full_fp(kperp, avg_bessel_factor, fp):
    #Sets grid size of velocity space integration.
    nv = 120

    #Initializes array for vperp and vperp_mid. 
    vperp = np.linspace(0, 6, nv, endpoint = True)
    vperp_mid = np.empty(nv-1)

    #Determines step size dv. 
    dv = vperp[1] - vperp[0]

    #Populates vperp_mid.
    for i in range(nv - 1):
        vperp_mid[i] = (vperp[i + 1] + vperp[i])/2.

    #Defines bessel factor for integral.
    j0m = j0(kperp * vperp_mid)

    #Performs integration. 
    numer_intv = 0.
    for j in range(nv-1):
        numer_intv += dv * vperp_mid[j] * np.exp(-vperp_mid[j]**2/2.) * j0m[j]**2 * (fp + (1 - fp) * (vperp_mid[j]**2/2. - 1.))

    return numer_intv - avg_bessel_factor


In [19]:
def bisection_fp(a, b, x0, tol, fp):
    """ 
    Uses the bisection method to solve for a root of the bessel function.
    """
    c = (a+b)/2.0
    while (b-a)/2.0 > tol:
        if bessel_full_fp(c, x0, fp) == 0:
            return c
        elif (bessel_full_fp(a, x0, fp) * bessel_full_fp(c, x0, fp)) < 0:
            b = c
        else:
            a = c
        c = (a+b)/2.0

    return c

In [20]:
def avg_omegad_bessel_full_omn_omt(z_grid,jacobian,kperp,omega_d,field,kperp_eff, omn, omt):
    """
    Calculates the eigenfunction-average omega_d considering fp. 
    """
    #Sets up grid points for the integration.
    nz = len(field)
    nv = 120
    vperp = np.linspace(0, 6, nv, endpoint = True)
    dv = vperp[1] - vperp[0]
    vperp_mid = np.empty(nv - 1)
    for j in range(nv - 1):
        vperp_mid[j] = (vperp[j + 1] + vperp[j]) * 0.5

    j0_m = np.empty((nz, nv - 1), dtype = 'float')
    for i in range(nz):
            j0_m[i, :] = j0(kperp[i] * vperp_mid)

    integrand_numer = np.empty((nz - 1, nv - 1), dtype = 'float')

    #Definition of fp.
    fp = float(omn) / (float(omn) + float(omt))

    #Calculation of the integral.
    #Numerator calculation.
    for i in range(nz - 1):
        for j in range(nv - 1):
            integrand_numer[i, j] = 0.5*(fp * (1. + vperp_mid[j]**2/2.) + (1. - fp) * vperp_mid[j]**4/4.) * \
                                    (omega_d[i] * abs(j0_m[i, j] * field[i])**2 + \
                                     omega_d[i+1] * abs(j0_m[i+1, j] * field[i+1])**2)

    numerz = np.empty(nz - 1, dtype = 'float')
    numer = 0.
    denom_intz = 0.

    for i in range(nz - 1):
        numer_intv = 0.
        for j in range(nv - 1):
            numer_intv += dv * vperp_mid[j] * np.exp(- vperp_mid[j]**2/2.) \
                           * integrand_numer[i, j]

        numer += (z_grid[i+1]-z_grid[i])/jacobian[i] * numer_intv
        denom_intz += (z_grid[i+1]-z_grid[i])/jacobian[i] * 0.5 * (abs(field[i])**2 + \
                                                            abs(field[i+1])**2)
        numerz[i] = numer_intv

    omdj0z = np.empty(nz - 1, dtype = 'float')
    for i in range(1, nz - 1):
        omdj0z[i] = (numerz[i] + numerz[i-1])*4./(abs(field[i-1])**2 + 2.*abs(field[i])**2 + abs(field[i+1])**2)
    j0_kperpeff = j0(kperp_eff * vperp_mid)

    #Denominator calculation.
    integrand_denom = np.empty(nv -1, dtype = 'float')
    
    for j in range(nv - 1):
        integrand_denom[j] = (fp * (1. + vperp_mid[j]**2/2.) + (1. - fp) * vperp_mid[j]**4/4.)*(j0_kperpeff[j])**2 
        
    denom_intv = 0.
    for j in range(nv - 1):
            denom_intv += dv * vperp_mid[j] * np.exp(- vperp_mid[j]**2/2.) * integrand_denom[j]


    denom = denom_intz * denom_intv

    ave_omegad = numer / denom

    ave_omegad_j0 = numer / (denom_intz * 2)

    return ave_omegad, ave_omegad_j0

In [22]:
def kz_from_dfielddz_bessel_full_omn_omt(kperp_eff, kperp, zgrid, jacobian, field, omn, omt, zstart, zend):
    """ 
    Calculates the eigenfunction average of kz. 
    """
    #Initializes number of points in z space.
    nz = len(field)

    #Defines fp.
    fp = float(omn) / (float(omn) + float(omt))

    #Calculates dfielddz.
    dfielddz = np.empty(len(field),dtype='complex128')
    for i in range(len(field)-1):
        dfielddz[i] = (field[i+1] - field[i])/\
                      (zgrid[i+1] - zgrid[i])*jacobian[i]

    if zstart == zend:
        zstart = float(raw_input("Enter start z: "))
        zend = float(raw_input("Enter end z: "))
    startInd = np.argmin(abs(zgrid - zstart))
    endInd = np.argmin(abs(zgrid - zend))

    #Initializes number of points in velocity space.
    nv = 120
    vperp = np.linspace(0, 6, nv, endpoint = True)
    dv = vperp[1] - vperp[0]
    vperp_mid = np.empty(nv - 1)
    for j in range(nv - 1):
        vperp_mid[j] = (vperp[j + 1] + vperp[j]) * 0.5

    #Bessel function term for values of vperp_mid.
    j0_m = np.empty((nz, nv - 1), dtype = 'float')
    for i in range(nz):
            j0_m[i, :] = j0(kperp[i] * vperp_mid)

    j0_kperpeff = j0(kperp_eff * vperp_mid)
    
    #Initializes the numerator and denominator term of the eigen-avg.
    integrand_numer = np.empty((nz - 1, nv - 1), dtype = 'float')
    integrand_denom = np.empty((nz - 1, nv - 1), dtype = 'float')

    #Performs the integral.
    for i in range(nz - 1):
        for j in range(nv - 1):
            integrand_numer[i, j] = abs((j0_m[i+1,j] * field[i+1]-\
                                         j0_m[i, j] * field[i] ) / \
                                         (zgrid[i+1] - zgrid[i]) * jacobian[i])**2 * \
                                         (fp + (1. - fp) * vperp_mid[j]**2/2.)
    for i in range(nz - 1):
        for j in range(nv - 1):
            integrand_denom[i, j] = 0.5*(abs(j0_kperpeff[j] * field[i])**2 + \
                                         abs(j0_kperpeff[j] * field[i+1])**2) * \
                                         (fp + (1. - fp) * vperp_mid[j]**2/2.)
    numer = 0.
    denom = 0.

    for i in range(startInd, endInd ):
        numer_intv = 0.
        denom_intv = 0.
        for j in range(nv - 1):
            numer_intv += dv * vperp_mid[j] * np.exp(- vperp_mid[j]**2/2.) \
                           * integrand_numer[i, j]

            denom_intv += dv * vperp_mid[j] * np.exp(- vperp_mid[j]**2/2.) \
                           * integrand_denom[i, j]

        numer += (zgrid[i+1]-zgrid[i])/jacobian[i] * numer_intv
        denom += (zgrid[i+1]-zgrid[i])/jacobian[i] * denom_intv

    ave_kz = np.sqrt(numer/denom)

    return ave_kz